# Ref-B
* GCR = 0.4
* Albedo = 0.2
* Hub Height = 3.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* For a 2-Up portrait system with a GCR of 0.4 the pitch is 2*5.96m = 11.92m
* 2-up Portrait: 5 rows, each with 25 x 2 = 50 modules (Figure 4)
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/Ref-B'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/Ref-B


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+237.g204310a.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'Ref-B'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1 # m
module_y = 2 # m. slope we will measure
sensorsy=2
torquetube = 0.15

# SceneDict Parameters
pitch = 11.92 # m
albedo = 0.2
hub_height = 3.5 # m  
nMods = 50 
nRows = 5

sceneDict = {'tilt':tilt,'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/Ref-B
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [6]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y)


Module Name: PVmod
Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [7]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.4,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [8]:
file = 'Phase2_meteo_hourly_psm3format.csv'
directory = os.path.expanduser('~/Documents/IEA-Bifacial-Tracking-Modeling')
weatherfile = os.path.join(directory, file)

metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-01-01_1000', endtime='2022-01-01_1200')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [9]:
trackerdict = demo.set1axis(**trackerParams)

In [10]:
if cumulativesky:
    demo.genCumSky1axis()
else:
    demo.gendaylit1axis()

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/


In [11]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/

Making 2 octfiles in root directory.
Created 1axis_2022-01-01_1030.oct
Created 1axis_2022-01-01_1130.oct


In [ ]:
trakerdict = demo.analysis1axis(sensorsy=2)

Linescan in process: 1axis_2022-01-01_1030_Row3_Module25_Front
Linescan in process: 1axis_2022-01-01_1030_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-01_1030_Row3_Module25.csv
Index: 2022-01-01_1030. Wm2Front: 102.7064. Wm2Back: 20.857999999999997
Linescan in process: 1axis_2022-01-01_1130_Row3_Module25_Front


In [ ]:
trakerdict

In [ ]:
demo

In [ ]:
demo.calculateResults()

In [ ]:
demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel